# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!
  
- 🤝 Breakout Room #2:
  - Part 1:
    1. Creating an Evaluation Dataset
    2. Adding Evaluators
    3. Evaluating
  - Part 2:
    1. Adding conditional check.

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effetively allowing us to recreate appliation flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [1]:
%pip install -U langchain langchain_openai langgraph arxiv duckduckgo-search

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.6 MB 15.9 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 32.7 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: duckduckgo-search
    Found existing installation: duckduckgo_search 5.3.1
    Uninstalling duckduckgo_search-5.3.1:
      Successfully uninstalled duckduckgo_search-5.3.1
Note: you may need to restart the kernel to use updated packages.


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE2 - LangGraph - {uuid4().hex[0:8]}"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_community.tools.tavily_search.tool import TavilySearchResults

tool_belt = [
    # DuckDuckGoSearchRun(),
    TavilySearchResults(),
    ArxivQueryRun()
]

### Actioning with Tools

Now that we've created our tool belt - we need to create a process that will let us leverage them when we need them.

We'll use the built-in [`ToolExecutor`](https://github.com/langchain-ai/langgraph/blob/main/langgraph/prebuilt/tool_executor.py) to do so.

In [5]:
from langgraph.prebuilt import ToolExecutor

tool_executor = ToolExecutor(tool_belt)

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [7]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

#### ❓ Question #1:

How does the model determine which tool to use?

#### ANSWER:

Both the tools we use extend the `langchain_core.tools.BaseTool` class, so they have a `description` attribute that describes the tool in natural language for the LLM to understand and determine usage.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolInvocation
import json
from langchain_core.messages import FunctionMessage

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `call_tool` is a node which will call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
workflow.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
workflow.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [14]:
app = workflow.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

#### ANSWER:

There shouldn't be any specific limit to the cycling, outside of practical bounds like the OpenAI API cost/rate limits, local compute resources, etc.

To impose a limit on the number of cycles, we could track the length of the `state["messages"]` list in our `should_continue` function, and return "END" once it crossed that limit (even before checking for "function_call" in the `additional_kwargs`).

#### Helper Function to print messages

In [15]:
def print_messages(messages):
  next_is_tool = False
  initial_query = True
  for message in messages["messages"]:
    if "function_call" in message.additional_kwargs:
      print()
      print(f'Tool Call - Name: {message.additional_kwargs["function_call"]["name"]} + Query: {message.additional_kwargs["function_call"]["arguments"]}')
      next_is_tool = True
      continue
    if next_is_tool:
      print(f"Tool Response: {message.content}")
      next_is_tool = False
      continue
    if initial_query:
      print(f"Initial Query: {message.content}")
      print()
      initial_query = False
      continue
    print()
    print(f"Agent Response: {message.content}")


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [18]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="What is RAG in the context of Large Language Models? When did it break onto the scene?")]}

messages = app.invoke(inputs)

print_messages(messages)

Initial Query: What is RAG in the context of Large Language Models? When did it break onto the scene?


Agent Response: RAG stands for Retrieval-Augmented Generation. It is a technique used in Large Language Models to improve the generation of text by combining retrieval-based methods with generative models. RAG models use a retriever to search through a large database of text to find relevant information that can be used to enhance the generation process of the language model.

RAG broke onto the scene in 2020 when a research paper titled "Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks" was published by researchers at Facebook AI. This paper introduced the concept of RAG and demonstrated its effectiveness in improving the performance of large language models on knowledge-intensive natural language processing tasks. Since then, RAG has gained popularity in the field of natural language processing and has been used in various applications to enhance the capabilities of

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "function_call" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "function_call" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [19]:
inputs = {"messages" : [HumanMessage(content="What is QLoRA in Machine Learning? Are their any technical papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?")]}

messages = app.invoke(inputs)

print_messages(messages)

Initial Query: What is QLoRA in Machine Learning? Are their any technical papers that could help me understand? Once you have that information, can you look up the bio of the first author on the QLoRA paper?


Tool Call - Name: arxiv + Query: {"query":"QLoRA in Machine Learning"}
Tool Response: Published: 2023-05-23
Title: QLoRA: Efficient Finetuning of Quantized LLMs
Authors: Tim Dettmers, Artidoro Pagnoni, Ari Holtzman, Luke Zettlemoyer
Summary: We present QLoRA, an efficient finetuning approach that reduces memory usage
enough to finetune a 65B parameter model on a single 48GB GPU while preserving
full 16-bit finetuning task performance. QLoRA backpropagates gradients through
a frozen, 4-bit quantized pretrained language model into Low Rank
Adapters~(LoRA). Our best model family, which we name Guanaco, outperforms all
previous openly released models on the Vicuna benchmark, reaching 99.3% of the
performance level of ChatGPT while only requiring 24 hours of finetuning on a
single GPU

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

#### ANSWER:

1) It first called the Arxiv tool to search for "QLoRA in Machine Learning"
2) It then identified "QLoRA: Efficient Finetuning of Quantized LLMs" as the QLoRA paper and picked up the name of its first author "Tim Dettmers"
3) Next, it called the Tavily tool to search for the bio of the first author using the query "Tim Dettmers bio"
4) Finally it returned a response which described QLoRA, gave the title and authors of the QLoRA paper, and returned information on the author's bio

# 🤝 Breakout Room #2

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [20]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | app | parse_output

In [21]:
agent_chain.invoke({"question" : "What is RAG?"})

'I am unable to retrieve search results for RAG at the moment. Would you like me to try searching for something else or provide information on a different topic?'

## Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

#### ANSWER:

The custom dataset is given below (the original dataset from the markdown was used for the actual notebook execution, but my custom data is available as commented code)

In [22]:
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
    # "What is the name of the paper that introduced the Transformer architecture in deep learning?",
    # "Who is the first author of the Attention Is All You Need paper?",
    # "What is the name of the GPT-3 paper?",
    # "What is the name of the paper introducing the MMLU benchmark?",
    # "Who is the first author of the Measuring Massive Multitask Language Understanding paper?",
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
    # {"must_mention" : ["Attention", "Is", "All", "You", "Need"]},
    # {"must_mention" : ["Ashish", "Vaswani"]},
    # {"must_mention" : ["Language", "Models", "are", "Few-Shot", "Learners"]},
    # {"must_mention" : ["Measuring", "Massive", "Multitask", "Language", "Understanding"]},
    # {"must_mention" : ["Dan", "Hendrycks"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [23]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

#### ANSWER:

The `Client.create_examples()` method zips the `questions` object with the `answers` object and creates the association between the two.

## Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [24]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

#### ANSWER:
Some ways this might be improved are:
1) We could update some of the `must_mention` word lists in the `answers` object to be more accurate to the respective questions (for example the "What is a Retrieval Augmented Generation system?" answer could also include "data source").
2) We could change the scoring logic, instead of using `score=all()` which requires all those exact match values to be retuned in the response (this would also require some updates to the answers).
3) We could use a full sentence answers in the `answers` object, and then use semantic matching through a method like cosine similarity.

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it, and a few others:

- `"criteria"` includes the default criteria which, in this case, means "helpfulness"
- `"cot_qa"` includes a criteria that bases whether or not the answer is correct by utilizing a Chain of Thought prompt and the provided context to determine if the response is correct or not.

In [25]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
    evaluators=[
        "criteria",
        "cot_qa",
    ],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [26]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 90ab1674' at:
https://smith.langchain.com/o/7fd07b5c-2cc4-5fa5-998d-744414e86be0/datasets/3e2c3f9f-bb84-490c-8404-f300bdeeaba2/compare?selectedSessions=f8976b36-9fb7-459a-8d34-2c34dffdf37c

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - b8c0d350 at:
https://smith.langchain.com/o/7fd07b5c-2cc4-5fa5-998d-744414e86be0/datasets/3e2c3f9f-bb84-490c-8404-f300bdeeaba2
[------------------------------------------------->] 6/6

,feedback.helpfulness,feedback.COT Contextual Accuracy,feedback.must_mention,error,execution_time,run_id
count,6.000000,5.000000,6,0,6.000000,6
unique,NaN,NaN,2,0,NaN,6
top,NaN,NaN,True,NaN,NaN,9f806170-bd75-4e26-9b0f-ec6da8982389
freq,NaN,NaN,3,NaN,NaN,1
mean,0.833333,0.800000,NaN,NaN,3.481814,NaN
std,0.408248,0.447214,NaN,NaN,0.997515,NaN
min,0.000000,0.000000,NaN,NaN,1.647430,NaN
25%,1.000000,1.000000,NaN,NaN,3.363078,NaN
50%,1.000000,1.000000,NaN,NaN,3.704475,NaN
75%,1.000000,1.000000,NaN,NaN,3.942334,NaN


{'project_name': 'RAG Pipeline - Evaluation - 90ab1674',
 'results': {'bbd7631e-b2ea-4d11-8b07-27c8bc28cfe5': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment='The criterion for this task is "helpfulness". \n\nThe submission provides a direct answer to the question asked in the input, which is about the optimizer used in QLoRA. The answer is specific and includes additional information about how to access the optimizer, which could be useful for someone trying to use or understand QLoRA. \n\nTherefore, the submission can be considered helpful, insightful, and appropriate. \n\nY', correction=None, evaluator_info={'__run': RunInfo(run_id=UUID('0685864a-fccb-45e3-8310-8bfe38c1a926'))}, feedback_config=None, source_run_id=None, target_run_id=None),
    EvaluationResult(key='COT Contextual Accuracy', score=None, value=None, comment='The context provided does not give any information about QLoRA or its

## Part 2: Helpfulness Check

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add a custom node and conditional edge to determine if the response was helpful enough.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [27]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

We're going to add a custom helpfulness check here!

In [35]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def check_helpfulness(state):
  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  # Added by Yohan
  print("Helpfulness Check: State Messages Length =", len(state["messages"]))

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    print("Helpful!\n")
    return "end"
  else:
    print("Not helpful!\n")
    return "continue"

def dummy_node(state):
  return

#### 🏗️ Activity #3:

Please write what is happening in our `check_helpfulness` function!

#### ANSWER:

In the `check_helpfulness` function, the folowing steps are executed:
1) We first create a prompt template from the `initial_query` and the `final_response` messages, and also instruct the model to indicate the helpfulness of the final response as it relates to the initial query
2) We then instantiate our GPT-4 evaluator model and create a chain using the prompt template, model and an output parser
3) Finally we invoke the chain with the `initial_query` and `final_response` messages, and then pass either "end" or "continue" depending on whether the model was helpful or not. This helps the agent graph decide whether to continue the cycle or terminate it and return the final response to the user.

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

In [36]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", call_tool)
graph_with_helpfulness_check.add_node("passthrough", dummy_node)

In [37]:
graph_with_helpfulness_check.set_entry_point("agent")

In [38]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : "passthrough"
    }
)

graph_with_helpfulness_check.add_conditional_edges(
    "passthrough",
    # should_continue,
    # Added by Yohan
    check_helpfulness,
    {
        "continue" : "agent",
        "end" : END
    }
)

In [39]:
graph_with_helpfulness_check.add_edge("action", "agent")

Let's compile and test!

In [40]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

In [41]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

messages = agent_with_helpfulness_check.invoke(inputs)

print_messages(messages)

Helpfulness Check: State Messages Length = 8
Helpful!
Initial Query: Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?


Tool Call - Name: tavily_search_results_json + Query: {"query":"LoRA in machine learning"}
Tool Response: [{'url': 'https://medium.com/data-science-in-your-pocket/lora-for-fine-tuning-llms-explained-with-codes-and-example-62a7ac5a3578', 'content': 'DBS Bank | Youtube: https://www.youtube.com/channel/UCQoNosQTIxiMTL9C-gvFdjA\nMore from Mehul Gupta and Data Science in your pocket\nMehul Gupta\nin\nData Science in your pocket\nRecommendation Systems using Langchain and LLMs with codes\nusing RAG framework and chains\n--\n3\nMehul Gupta\nin\nData Science in your pocket\nLangchain tutorials for newbies\nLangchain use cases with demo explained\n--\n2\nMehul Gupta\nin\nData Science in your pocket\nWhat are Vector Databases and How Langchain uses Vector DBs\nwith codes and examples\n--\n1\nMehul Gupta\nin\nData Science in your poc